<a href="https://colab.research.google.com/github/smalaboy/applied-data-science-projects/blob/main/flant_t5/flan_t5_finetuning_for_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip '/content/archive (8).zip'

Archive:  /content/archive (8).zip
  inflating: IMDB Dataset.csv        


In [ ]:
!pip install -q transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

tqdm.pandas()

In [ ]:
data_path = "/content/IMDB Dataset.csv"
RANDOM_STATE = 42

In [ ]:
df = pd.read_csv(data_path)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.describe(include='all')

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [ ]:
df.groupby('sentiment').count()

,review
sentiment,
negative,25000
positive,25000


In [ ]:
train_idx = df.sample(frac=0.7, random_state=RANDOM_STATE).index
test_val_df = df.drop(train_idx)
test_idx = test_val_df.sample(frac=0.5, random_state=RANDOM_STATE).index
val_idx = test_val_df.drop(test_idx).index

df['split'] = 0
df['split'][test_idx] = 1
df['split'][val_idx] = 2

<ipython-input-7-065d3554b389>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['split'][test_idx] = 1
<ipython-input-7-065d3554b389>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['split'][val_idx] = 2


In [ ]:
df.head(10)

,review,sentiment,split
0,One of the other reviewers has mentioned that ...,positive,0
1,A wonderful little production. <br /><br />The...,positive,0
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,0
5,"Probably my all-time favorite movie, a story o...",positive,2
6,I sure would like to see a resurrection of a u...,positive,0
7,"This show was an amazing, fresh & innovative i...",negative,0
8,Encouraged by the positive comments about this...,negative,0
9,If you like original gut wrenching laughter yo...,positive,2


In [ ]:
df.groupby(['sentiment', 'split']).count()

review
sentiment split        
negative  0       17517
          1        3741
          2        3742
positive  0       17483
          1        3759
          2        3758

In [ ]:
possible_questions = [
    "What's the sentiment of this review?",
    "How is this review ?",
    "Find the sentiment in this review.",
    "Is the review positive or negative?"
]

def make_prompt(row):
  review = row.loc['review']
  sentiment = row.loc['sentiment']
  question = possible_questions[random.randint(0, len(possible_questions) -1 )]
  prompt = f"""
  {question}

  Review:
  {review}

  Sentiment:
  """

  return prompt

In [ ]:
df['prompt'] = df.apply(make_prompt, axis=1)

In [ ]:
df.head()

,review,sentiment,split,prompt
0,One of the other reviewers has mentioned that ...,positive,0,\n Is the review positive or negative?\n\n R...
1,A wonderful little production. <br /><br />The...,positive,0,\n What's the sentiment of this review?\n\n ...
2,I thought this was a wonderful way to spend ti...,positive,1,\n Find the sentiment in this review.\n\n Re...
3,Basically there's a family where a little boy ...,negative,0,\n Find the sentiment in this review.\n\n Re...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,0,\n How is this review ?\n\n Review:\n Pette...


## Loading dataset and LLM

In [ ]:
train_dataset = Dataset.from_pandas(df[df['split'] == 0])
val_dataset = Dataset.from_pandas(df[df['split'] == 1])
test_dataset = Dataset.from_pandas(df[df['split'] == 2])

dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

dataset

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment', 'split', 'prompt', '__index_level_0__'],
        num_rows: 35000
    })
    validation: Dataset({
        features: ['review', 'sentiment', 'split', 'prompt', '__index_level_0__'],
        num_rows: 7500
    })
    test: Dataset({
        features: ['review', 'sentiment', 'split', 'prompt', '__index_level_0__'],
        num_rows: 7500
    })
})

In [ ]:
model_name='google/flan-t5-small'
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 76961152
all model parameters: 76961152
percentage of trainable model parameters: 100.00%


## Zero shot inferencing

In [ ]:
idx = 0
example_review = dataset['test'][idx]['review']
example_review_sentiment = dataset['test'][idx]['sentiment']
prompt = f"""
What's the sentiment of this review?

Review:
{example_review}

Sentiment:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(inputs['input_ids'], max_new_tokens = 200)[0],
    skip_special_tokens = True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN CLASSIFICATION:\n{example_review_sentiment}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

What's the sentiment of this review?

Review:
Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it's not preachy or boring. It just never gets old, despite my having seen it some 15 or more times in the last 25 years. Paul Lukas' performance brings tears to my eyes, and Bette Davis, in one of her very few truly sympathetic roles, is a delight. The kids are, as grandma says, more like "dressed-up midgets" than children, but that only makes them more fun to watch. And the mother's slow awakening to what's happening in the world and under her own roof is believable and startling. If I had a dozen thumbs, they'd all be "up" for this movie.

Sentiment:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
positive

-----------------------------------

## One shot inference

In [ ]:
idx = 0
idx2 = 235
example_review = dataset['test'][idx]['review']
example_review_sentiment = dataset['test'][idx]['sentiment']

example_review2 = dataset['test'][idx2]['review']
example_review_sentiment2 = dataset['test'][idx2]['sentiment']

prompt = f"""
What's the sentiment of this review?

Review:
{example_review2}

Sentiment:
{example_review_sentiment2}

What's the sentiment of this review?

Review:
{example_review}

Sentiment:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(inputs['input_ids'], max_new_tokens = 200)[0],
    skip_special_tokens = True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN CLASSIFICATION:\n{example_review_sentiment}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

What's the sentiment of this review?

Review:
A found tape about 3? guys having fun torturing a woman in several inhuman ways.<br /><br />Yeah, spoiler.<br /><br />First of all, the acting made this short not scary at all, the woman seemed to have orgasms, not suffering. Some of the punishments were so ridiculous! what's shocking about throwing some meat or spin her in a chair? If you are shooting a nonsense tape, at least make it good. The only part to remark is the end: the hammered hand and the pierced eye, the rest of the film is really poor. To end the boredom, the supposed story about the tape being investigated, extra bullshit.

Sentiment:
negative

What's the sentiment of this review?

Review:
Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it's not preachy or boring. It just never gets old, despite my ha